In [15]:
import time

import numpy as np
import torch
from itertools import product
import src.utils as ut
from src.SCMMappings_1_1 import Abstraction
from src.printing import AbstractionPrinter

import matplotlib.pyplot as plt
from pgmpy.models import BayesianNetwork as BN
from pgmpy.factors.discrete import TabularCPD as cpd

In [16]:
seed = 0
np.random.seed(seed)  # Numpy module
torch.manual_seed(seed)

In [17]:
from src.examples import smokingmodels as ex

In [18]:
A_basic = Abstraction(*ex.basic_lung_cancer())
A_collapsing = Abstraction(*ex.collapsing_lung_cancer())
A_extended = Abstraction(*ex.extended_lung_cancer())
A_vstruct = Abstraction(*ex.vstruct_lung_cancer())


In [19]:
import src.evaluationsets as es
import src.learning as lng

In [20]:
J_basic = es.get_causal_sets_in_M1_with_directed_path_in_M1_and_M0(A_basic.M0, A_basic.M1, A_basic.a)

In [21]:
J_collapsing = [[['Smoking_'], ['Tar_']],
 [['Smoking_'], ['Cancer_']],
 [['Tar_'], ['Cancer_']]]

In [22]:
J_extended = [[['Smoking_'], ['Tar_']],
 [['Smoking_'], ['Cancer_']],
 [['Tar_'], ['Cancer_']]]

In [23]:
J_vstruct = [[['Smoking_'], ['Cancer_']],
 [['Smoking_'], ['Fatigue_']],
 [['Cancer_'], ['Fatigue_']]]

In [24]:
from src.nn import JointNeuralNet
from src.nn_losses import JSD_loss,RowMax_penalty

In [25]:
def calc_distance(Ws, alpha_index, alphas):
    distance = 0.0
    for key, value in alphas.items():
        distance += torch.sum(torch.abs(torch.from_numpy(value)-Ws[alpha_index[key]]))
    return distance

In [26]:
def train_a_JointNN_model(M0,M1,R,a,alphas,J,Ws,T,lmbda,num_epochs,lr):
    distances = np.zeros((num_epochs))
    jsds = np.zeros((num_epochs))
    pens = np.zeros((num_epochs))
    end_results = {}
    
    
    model = JointNeuralNet(M0,M1,R,a,J,T)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion_jsd = JSD_loss()
    criterion_pen = RowMax_penalty()
    
    
    for epoch in range(num_epochs):
        lowpaths, highpaths, Ws = model()    
        loss_jsd = criterion_jsd(lowpaths, highpaths)
        loss_pen = criterion_pen(Ws)
        loss = lmbda*loss_jsd + loss_pen
        
        distances[epoch] = calc_distance(Ws, model.alpha_index, alphas)
        jsds[epoch] = loss_jsd
        pens[epoch] = loss_pen

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (epoch+1) % 500 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}, JSD: {:.4f}, Penalty: {:.4f}'.format(epoch+1, num_epochs, loss, loss_jsd, loss_pen))
    
    learnt_alphas = {}
    for name in model.alphas_to_learn:
        learnt_alphas[name] = model.get_Wmatrix_by_name(name).detach()
        
    # discretize alphas and calc_distance
    discretized_alphas = {}
    for name, value in learnt_alphas.items():
        tmp = torch.zeros((value.shape))
        for i in range(value.shape[1]):
            tmp[torch.argmax(value[:,i]),i] = 1
        discretized_alphas[name] = tmp
    
    #calculate distance with discrete learnt alphas
    end_distance = 0.0
    for key, value in alphas.items():
        end_distance += torch.sum(torch.abs(torch.from_numpy(alphas[key])-discretized_alphas[key]))
    
    # jsd and penalty with initalised discrete alphas
    criterion_jsd = JSD_loss()
    criterion_pen = RowMax_penalty()
    end_model = JointNeuralNet(M0,M1,R,a,J,T,initialised_alphas=discretized_alphas)
    lowpaths, highpaths, Ws = end_model()    
    end_loss_jsd = criterion_jsd(lowpaths, highpaths)
    end_loss_pen = criterion_pen(Ws)
    
    # store end results
    end_results['normalised_alphas'] = learnt_alphas
    end_results['discretized_alpha'] = discretized_alphas
    end_results['jsd'] = end_loss_jsd
    end_results['penalty'] = end_loss_pen
    end_results['distance'] = end_distance
    
    return distances/len(alphas), jsds/len(J), pens/len(alphas), end_results

In [27]:
lr = 0.01
num_epochs = 500
Ts = [0.01, 0.05, 0.1, 0.5, 1.0]
lmbdas = [1.0, 5.0, 10.0, 20.0, 50.0]
n_fold = 10
k_fold = 10

In [28]:
prod = list(product(Ts,lmbdas))

for T, lmbda in prod:
    
    n_best_distance1 = np.zeros((n_fold,num_epochs))
    n_best_jsd1 = np.zeros((n_fold,num_epochs))
    n_best_pen1 = np.zeros((n_fold,num_epochs))
    n_best_end_results1 = []
    n_time1 = np.zeros((n_fold))

    n_best_distance2 = np.zeros((n_fold,num_epochs))
    n_best_jsd2 = np.zeros((n_fold,num_epochs))
    n_best_pen2 = np.zeros((n_fold,num_epochs))
    n_best_end_results2 = []
    n_time2 = np.zeros((n_fold))

    n_best_distance3 = np.zeros((n_fold,num_epochs))
    n_best_jsd3 = np.zeros((n_fold,num_epochs))
    n_best_pen3 = np.zeros((n_fold,num_epochs))
    n_best_end_results3 = []
    n_time3 = np.zeros((n_fold))

    n_best_distance4 = np.zeros((n_fold,num_epochs))
    n_best_jsd4 = np.zeros((n_fold,num_epochs))
    n_best_pen4 = np.zeros((n_fold,num_epochs))
    n_best_end_results4 = []
    n_time4 = np.zeros((n_fold))


    for n in range(n_fold):
        print(f"n fold {n+1}:")

        k_distances1 = np.zeros((k_fold,num_epochs))
        k_jsds1 = np.zeros((k_fold,num_epochs))
        k_pens1 = np.zeros((k_fold,num_epochs))
        k_end_results1 = []
        k_time1 = np.zeros((k_fold))

        k_distances2 = np.zeros((k_fold,num_epochs))
        k_jsds2 = np.zeros((k_fold,num_epochs))
        k_pens2 = np.zeros((k_fold,num_epochs))
        k_end_results2 = []
        k_time2 = np.zeros((k_fold))

        k_distances3 = np.zeros((k_fold,num_epochs))
        k_jsds3 = np.zeros((k_fold,num_epochs))
        k_pens3 = np.zeros((k_fold,num_epochs))
        k_end_results3 = []
        k_time3 = np.zeros((k_fold))

        k_distances4 = np.zeros((k_fold,num_epochs))
        k_jsds4 = np.zeros((k_fold,num_epochs))
        k_pens4 = np.zeros((k_fold,num_epochs))
        k_end_results4 = []
        k_time4 = np.zeros((k_fold))

        for fold in range(k_fold):
            st = time.time()
            distances_epoch1, jsds1, pens1, end_results1 = train_a_JointNN_model(A_basic.M0, A_basic.M1, A_basic.R, A_basic.a, A_basic.alphas, J_basic,
                         Ws=None, T=T, lmbda=lmbda, num_epochs=num_epochs, lr=lr)
            et = time.time()
            time1 = et - st

            st = time.time()
            distances_epoch2, jsds2, pens2, end_results2 = train_a_JointNN_model(A_collapsing.M0, A_collapsing.M1, A_collapsing.R, A_collapsing.a, A_collapsing.alphas, J_collapsing,
                         Ws=None, T=T, lmbda=lmbda, num_epochs=num_epochs, lr=lr)
            et = time.time()
            time2 = et - st

            st = time.time()
            distances_epoch3, jsds3, pens3, end_results3 = train_a_JointNN_model(A_extended.M0, A_extended.M1, A_extended.R, A_extended.a, A_extended.alphas, J_extended,
                         Ws=None, T=T, lmbda=lmbda, num_epochs=num_epochs, lr=lr)
            et = time.time()
            time3 = et - st

            st = time.time()
            distances_epoch4, jsds4, pens4, end_results4 = train_a_JointNN_model(A_vstruct.M0, A_vstruct.M1, A_vstruct.R, A_vstruct.a, A_vstruct.alphas, J_vstruct,
                         Ws=None, T=T, lmbda=lmbda, num_epochs=num_epochs, lr=lr)
            et = time.time()
            time4 = et - st


            k_distances1[fold] = distances_epoch1
            k_jsds1[fold] = jsds1
            k_pens1[fold] = pens1
            k_end_results1.append(end_results1)
            k_time1[fold] = time1

            k_distances2[fold] = distances_epoch2
            k_jsds2[fold] = jsds2
            k_pens2[fold] = pens2
            k_end_results2.append(end_results2)
            k_time2[fold] = time2

            k_distances3[fold] = distances_epoch3
            k_jsds3[fold] = jsds3
            k_pens3[fold] = pens3
            k_end_results3.append(end_results3)
            k_time3[fold] = time3

            k_distances4[fold] = distances_epoch4
            k_jsds4[fold] = jsds4
            k_pens4[fold] = pens4
            k_end_results4.append(end_results4)
            k_time4[fold] = time4

        k_losses1 = k_jsds1+k_pens1
        k_losses2 = k_jsds2+k_pens2
        k_losses3 = k_jsds3+k_pens3
        k_losses4 = k_jsds4+k_pens4
        best1 = np.argmin(k_losses1[:,-1])
        best2 = np.argmin(k_losses2[:,-1])
        best3 = np.argmin(k_losses3[:,-1])
        best4 = np.argmin(k_losses4[:,-1])

        n_best_distance1[n] = k_distances1[best1]
        n_best_jsd1[n] = k_jsds1[best1]
        n_best_pen1[n] = k_pens1[best1]
        n_best_end_results1.append(k_end_results1[best1])
        n_time1[n] = k_time1[best1]

        n_best_distance2[n] = k_distances2[best2]
        n_best_jsd2[n] = k_jsds2[best2]
        n_best_pen2[n] = k_pens2[best2]
        n_best_end_results2.append(k_end_results2[best2])
        n_time2[n] = k_time2[best2]

        n_best_distance3[n] = k_distances3[best3]
        n_best_jsd3[n] = k_jsds3[best3]
        n_best_pen3[n] = k_pens3[best3]
        n_best_end_results3.append(k_end_results3[best3])
        n_time3[n] = k_time3[best3]

        n_best_distance4[n] = k_distances4[best4]
        n_best_jsd4[n] = k_jsds4[best4]
        n_best_pen4[n] = k_pens4[best4]
        n_best_end_results4.append(k_end_results4[best4])
        n_time4[n] = k_time4[best4]
        

    avg_dis1 = np.mean(n_best_distance1,axis=0)
    avg_dis_err1 = np.std(n_best_distance1,axis=0)

    avg_jsd1 = np.mean(n_best_jsd1,axis=0)
    avg_jsd_err1 = np.std(n_best_jsd1,axis=0)

    avg_pen1 = np.mean(n_best_pen1,axis=0)
    avg_pen_err1 = np.std(n_best_pen1,axis=0)

    avg_time1 = np.mean(n_time1,axis=0)
    avg_time_err1 = np.std(n_time1,axis=0)
    print("======================================")
    print(f"T: {str(T)} Lambda: {str(lmbda)}")
    print(f"Basic: Avg Alpha distance: {avg_dis1[-1]}")
    print(f"Basic: Avg JSD: {avg_jsd1[-1]}")
    print(f"Basic: Avg Penalty: {avg_pen1[-1]}")
    print(f"Basic: Avg Time: {avg_time1}")

    print(f"Basic: Avg Alpha distance SD: {avg_dis_err1[-1]}")
    print(f"Basic: Avg JSD SD: {avg_jsd_err1[-1]}")
    print(f"Basic: Avg Penalty SD: {avg_pen_err1[-1]}")
    print(f"Basic: Avg Time SD: {avg_time_err1}")


    avg_dis2 = np.mean(n_best_distance2,axis=0)
    avg_dis_err2 = np.std(n_best_distance2,axis=0)

    avg_jsd2 = np.mean(n_best_jsd2,axis=0)
    avg_jsd_err2 = np.std(n_best_jsd2,axis=0)

    avg_pen2 = np.mean(n_best_pen2,axis=0)
    avg_pen_err2 = np.std(n_best_pen2,axis=0)

    avg_time2 = np.mean(n_time2,axis=0)
    avg_time_err2 = np.std(n_time2,axis=0)

    print(f"Collapsing: Avg Alpha distance: {avg_dis2[-1]}")
    print(f"Collapsing: Avg JSD: {avg_jsd2[-1]}")
    print(f"Collapsing: Avg Penalty: {avg_pen2[-1]}")
    print(f"Collapsing: Avg Time: {avg_time2}")

    print(f"Collapsing: Avg Alpha distance SD: {avg_dis_err2[-1]}")
    print(f"Collapsing: Avg JSD SD: {avg_jsd_err2[-1]}")
    print(f"Collapsing: Avg Penalty SD: {avg_pen_err2[-1]}")
    print(f"Collapsing: Avg Time SD: {avg_time_err2}")


    avg_dis3 = np.mean(n_best_distance3,axis=0)
    avg_dis_err3 = np.std(n_best_distance3,axis=0)

    avg_jsd3 = np.mean(n_best_jsd3,axis=0)
    avg_jsd_err3 = np.std(n_best_jsd3,axis=0)

    avg_pen3 = np.mean(n_best_pen3,axis=0)
    avg_pen_err3 = np.std(n_best_pen3,axis=0)

    avg_time3 = np.mean(n_time3,axis=0)
    avg_time_err3 = np.std(n_time3,axis=0)

    print(f"Extended: Avg Alpha distance: {avg_dis3[-1]}")
    print(f"Extended: Avg JSD: {avg_jsd3[-1]}")
    print(f"Extended: Avg Penalty: {avg_pen3[-1]}")
    print(f"Extended: Avg Time: {avg_time3}")

    print(f"Extended: Avg Alpha distance SD: {avg_dis_err3[-1]}")
    print(f"Extended: Avg JSD SD: {avg_jsd_err3[-1]}")
    print(f"Extended: Avg Penalty SD: {avg_pen_err3[-1]}")
    print(f"Extended: Avg Time SD: {avg_time_err3}")


    avg_dis4 = np.mean(n_best_distance4,axis=0)
    avg_dis_err4 = np.std(n_best_distance4,axis=0)

    avg_jsd4 = np.mean(n_best_jsd4,axis=0)
    avg_jsd_err4 = np.std(n_best_jsd4,axis=0)

    avg_pen4 = np.mean(n_best_pen4,axis=0)
    avg_pen_err4 = np.std(n_best_pen4,axis=0)

    avg_time4 = np.mean(n_time4,axis=0)
    avg_time_err4 = np.std(n_time4,axis=0)

    print(f"Vstruct: Avg Alpha distance: {avg_dis4[-1]}")
    print(f"Vstruct: Avg JSD: {avg_jsd4[-1]}")
    print(f"Vstruct: Avg Penalty: {avg_pen4[-1]}")
    print(f"Vstruct: Avg Time: {avg_time4}")

    print(f"Vstruct: Avg Alpha distance SD: {avg_dis_err4[-1]}")
    print(f"Vstruct: Avg JSD SD: {avg_jsd_err4[-1]}")
    print(f"Vstruct: Avg Penalty SD: {avg_pen_err4[-1]}")
    print(f"Vstruct: Avg Time SD: {avg_time_err4}")
    
    # Save results
    np.save(f'./results/synthetic/d/avg-best-distance-app3-t{str(T)}-l{str(lmbda)}-basic', n_best_distance1)
    np.save(f'./results/synthetic/d/avg-best-JSD-app3-t{str(T)}-l{str(lmbda)}-basic', n_best_jsd1)
    np.save(f'./results/synthetic/d/avg-best-penalty-app3-t{str(T)}-l{str(lmbda)}-basic', n_best_pen1)
    np.save(f'./results/synthetic/d/avg-best-time-app3-t{str(T)}-l{str(lmbda)}-basic', n_time1)
    np.save(f'./results/synthetic/d/avg-best-endresults-app3-t{str(T)}-l{str(lmbda)}-basic', n_best_end_results1)

    np.save(f'./results/synthetic/d/avg-best-distance-app3-t{str(T)}-l{str(lmbda)}-collapsing', n_best_distance2)
    np.save(f'./results/synthetic/d/avg-best-JSD-app3-t{str(T)}-l{str(lmbda)}-collapsing', n_best_jsd2)
    np.save(f'./results/synthetic/d/avg-best-penalty-app3-t{str(T)}-l{str(lmbda)}-collapsing', n_best_pen2)
    np.save(f'./results/synthetic/d/avg-best-time-app3-t{str(T)}-l{str(lmbda)}-collapsing', n_time2)
    np.save(f'./results/synthetic/d/avg-best-endresults-app3-t{str(T)}-l{str(lmbda)}-collapsing', n_best_end_results2)

    np.save(f'./results/synthetic/d/avg-best-distance-app3-t{str(T)}-l{str(lmbda)}-extended', n_best_distance3)
    np.save(f'./results/synthetic/d/avg-best-JSD-app3-t{str(T)}-l{str(lmbda)}-extended', n_best_jsd3)
    np.save(f'./results/synthetic/d/avg-best-penalty-app3-t{str(T)}-l{str(lmbda)}-extended', n_best_pen3)
    np.save(f'./results/synthetic/d/avg-best-time-app3-t{str(T)}-l{str(lmbda)}-extended', n_time3)
    np.save(f'./results/synthetic/d/avg-best-endresults-app3-t{str(T)}-l{str(lmbda)}-extended', n_best_end_results3)

    np.save(f'./results/synthetic/d/avg-best-distance-app3-t{str(T)}-l{str(lmbda)}-vstruct', n_best_distance4)
    np.save(f'./results/synthetic/d/avg-best-JSD-app3-t{str(T)}-l{str(lmbda)}-vstruct', n_best_jsd4)
    np.save(f'./results/synthetic/d/avg-best-penalty-app3-t{str(T)}-l{str(lmbda)}-vstruct', n_best_pen4)
    np.save(f'./results/synthetic/d/avg-best-time-app3-t{str(T)}-l{str(lmbda)}-vstruct', n_time4)
    np.save(f'./results/synthetic/d/avg-best-endresults-app3-t{str(T)}-l{str(lmbda)}-vstruct', n_best_end_results4)

        
          
        

n fold 1:
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: 1.7353, JSD: 0.7353, Penalty: 1.0000
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/5

Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD

Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
n fold 9:
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: 0.2630, JSD: 0.2630, Penalty: 0.0000
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/5

Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: 5.1857, JSD: 0.8371, Penalty: 1.0000
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss:

Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: 2.2918, JSD: 0.4584, Penalty: 0.0000
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: nan, JSD: nan, Penalty: nan
Epoch [500/500], Loss: 1.7540, JSD: 0.3508, Penalty: 0.0000
Epoch [500/50

...

Epoch [500/500], Loss: 2.0651, JSD: 0.0017, Penalty: 2.0315
Epoch [500/500], Loss: 1.8253, JSD: 0.0020, Penalty: 1.7860
Epoch [500/500], Loss: 1.1402, JSD: 0.0014, Penalty: 1.1127
Epoch [500/500], Loss: 1.3154, JSD: 0.0004, Penalty: 1.3066
Epoch [500/500], Loss: 0.9777, JSD: 0.0015, Penalty: 0.9473
Epoch [500/500], Loss: 2.3681, JSD: 0.0025, Penalty: 2.3181
Epoch [500/500], Loss: 1.4494, JSD: 0.0020, Penalty: 1.4085
Epoch [500/500], Loss: 1.3298, JSD: 0.0010, Penalty: 1.3100
Epoch [500/500], Loss: 2.0919, JSD: 0.0016, Penalty: 2.0608
Epoch [500/500], Loss: 1.1651, JSD: 0.0018, Penalty: 1.1285
Epoch [500/500], Loss: 1.4436, JSD: 0.0014, Penalty: 1.4161
Epoch [500/500], Loss: 0.3636, JSD: 0.0004, Penalty: 0.3550
Epoch [500/500], Loss: 2.3880, JSD: 0.0023, Penalty: 2.3422
Epoch [500/500], Loss: 1.9226, JSD: 0.0020, Penalty: 1.8828
Epoch [500/500], Loss: 1.3108, JSD: 0.0014, Penalty: 1.2820
Epoch [500/500], Loss: 0.2204, JSD: 0.0004, Penalty: 0.2119
Epoch [500/500], Loss: 1.2805, JSD: 0.00

Epoch [500/500], Loss: 1.8999, JSD: 0.0018, Penalty: 1.8636
Epoch [500/500], Loss: 1.5457, JSD: 0.0014, Penalty: 1.5170
Epoch [500/500], Loss: 0.2247, JSD: 0.0004, Penalty: 0.2165
Epoch [500/500], Loss: 2.1566, JSD: 0.0022, Penalty: 2.1134
Epoch [500/500], Loss: 1.7690, JSD: 0.0016, Penalty: 1.7371
Epoch [500/500], Loss: 1.3073, JSD: 0.0014, Penalty: 1.2793
n fold 8:
Epoch [500/500], Loss: 0.2435, JSD: 0.0004, Penalty: 0.2360
Epoch [500/500], Loss: 1.7332, JSD: 0.0016, Penalty: 1.7008
Epoch [500/500], Loss: 1.4179, JSD: 0.0016, Penalty: 1.3864
Epoch [500/500], Loss: 1.4420, JSD: 0.0013, Penalty: 1.4155
Epoch [500/500], Loss: 1.3082, JSD: 0.0005, Penalty: 1.2990
Epoch [500/500], Loss: 0.7792, JSD: 0.0016, Penalty: 0.7476
Epoch [500/500], Loss: 1.1693, JSD: 0.0018, Penalty: 1.1332
Epoch [500/500], Loss: 1.5319, JSD: 0.0015, Penalty: 1.5028
Epoch [500/500], Loss: 0.2055, JSD: 0.0005, Penalty: 0.1963
Epoch [500/500], Loss: 1.3621, JSD: 0.0017, Penalty: 1.3288
Epoch [500/500], Loss: 2.2946,

Epoch [500/500], Loss: 1.4103, JSD: 0.0004, Penalty: 1.3902
Epoch [500/500], Loss: 1.0714, JSD: 0.0015, Penalty: 0.9944
Epoch [500/500], Loss: 2.7513, JSD: 0.0017, Penalty: 2.6640
Epoch [500/500], Loss: 1.8870, JSD: 0.0013, Penalty: 1.8243
Epoch [500/500], Loss: 1.3826, JSD: 0.0004, Penalty: 1.3603
Epoch [500/500], Loss: 1.8291, JSD: 0.0015, Penalty: 1.7541
Epoch [500/500], Loss: 1.3331, JSD: 0.0017, Penalty: 1.2502
Epoch [500/500], Loss: 1.6047, JSD: 0.0011, Penalty: 1.5511
Epoch [500/500], Loss: 1.4002, JSD: 0.0005, Penalty: 1.3748
Epoch [500/500], Loss: 1.8154, JSD: 0.0018, Penalty: 1.7238
Epoch [500/500], Loss: 2.2022, JSD: 0.0016, Penalty: 2.1241
Epoch [500/500], Loss: 1.2838, JSD: 0.0014, Penalty: 1.2134
Epoch [500/500], Loss: 1.5371, JSD: 0.0009, Penalty: 1.4930
Epoch [500/500], Loss: 1.5106, JSD: 0.0016, Penalty: 1.4318
Epoch [500/500], Loss: 2.1571, JSD: 0.0017, Penalty: 2.0729
Epoch [500/500], Loss: 1.6856, JSD: 0.0013, Penalty: 1.6203
Epoch [500/500], Loss: 1.4275, JSD: 0.00

Epoch [500/500], Loss: 1.9915, JSD: 0.0015, Penalty: 1.9143
Epoch [500/500], Loss: 2.0479, JSD: 0.0015, Penalty: 1.9704
Epoch [500/500], Loss: 1.7642, JSD: 0.0012, Penalty: 1.7027
Epoch [500/500], Loss: 1.3194, JSD: 0.0004, Penalty: 1.2997
Epoch [500/500], Loss: 1.7129, JSD: 0.0016, Penalty: 1.6306
Epoch [500/500], Loss: 1.1154, JSD: 0.0015, Penalty: 1.0386
Epoch [500/500], Loss: 1.9745, JSD: 0.0012, Penalty: 1.9146
Epoch [500/500], Loss: 0.5793, JSD: 0.0004, Penalty: 0.5578
Epoch [500/500], Loss: 2.1366, JSD: 0.0014, Penalty: 2.0656
Epoch [500/500], Loss: 1.0035, JSD: 0.0016, Penalty: 0.9258
Epoch [500/500], Loss: 1.5185, JSD: 0.0013, Penalty: 1.4551
Epoch [500/500], Loss: 0.5962, JSD: 0.0004, Penalty: 0.5737
Epoch [500/500], Loss: 2.2962, JSD: 0.0016, Penalty: 2.2151
Epoch [500/500], Loss: 2.7752, JSD: 0.0015, Penalty: 2.6996
Epoch [500/500], Loss: 1.5409, JSD: 0.0012, Penalty: 1.4802
Epoch [500/500], Loss: 1.4881, JSD: 0.0004, Penalty: 1.4664
Epoch [500/500], Loss: 2.3306, JSD: 0.00

Epoch [500/500], Loss: 2.3369, JSD: 0.0017, Penalty: 2.2524
Epoch [500/500], Loss: 1.3646, JSD: 0.0014, Penalty: 1.2941
Epoch [500/500], Loss: 1.4598, JSD: 0.0004, Penalty: 1.4415
Epoch [500/500], Loss: 2.2022, JSD: 0.0015, Penalty: 2.1281
Epoch [500/500], Loss: 2.7561, JSD: 0.0015, Penalty: 2.6802
Epoch [500/500], Loss: 1.5989, JSD: 0.0013, Penalty: 1.5347
n fold 8:
Epoch [500/500], Loss: 1.3915, JSD: 0.0005, Penalty: 1.3668
Epoch [500/500], Loss: 2.4776, JSD: 0.0018, Penalty: 2.3886
Epoch [500/500], Loss: 1.9722, JSD: 0.0017, Penalty: 1.8889
Epoch [500/500], Loss: 1.9795, JSD: 0.0012, Penalty: 1.9172
Epoch [500/500], Loss: 1.3594, JSD: 0.0004, Penalty: 1.3394
Epoch [500/500], Loss: 2.5136, JSD: 0.0017, Penalty: 2.4290
Epoch [500/500], Loss: 2.0335, JSD: 0.0016, Penalty: 1.9531
Epoch [500/500], Loss: 1.6343, JSD: 0.0013, Penalty: 1.5712
Epoch [500/500], Loss: 1.4659, JSD: 0.0004, Penalty: 1.4439
Epoch [500/500], Loss: 2.0744, JSD: 0.0016, Penalty: 1.9966
Epoch [500/500], Loss: 2.0100,